In [1]:
import tensorflow as tf
import tensorflow_text as text
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
from time import time



In [2]:
def recall(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        
        recall = TP / (Positives+K.epsilon())    
        return recall 

def precision(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
        precision = TP / (Pred_Positives+K.epsilon())
        return precision 

def f1_score(y_true, y_pred):    
    
    p_recision = precision(y_true, y_pred), 
    r_call= recall(y_true, y_pred)
    
    return 2*((p_recision*r_call)/(p_recision+r_call+K.epsilon()))

In [3]:
with open('Datasets/SNIPS/test/seq.in') as file:
    test_x = np.asarray([x.strip() for x in file])

with open('Datasets/SNIPS/test/label') as file:
    test_y = [x.strip() for x in file]

print("There are total {} texts in test dataset.".format(len(test_x)))

labels, label_count = np.unique(np.array(test_y),return_counts=True)
pd.DataFrame(list(zip(labels,label_count)),columns=['label','label_count'])

There are total 700 texts in test dataset.


,label,label_count
0,AddToPlaylist,124
1,BookRestaurant,92
2,GetWeather,104
3,PlayMusic,86
4,RateBook,80
5,SearchCreativeWork,107
6,SearchScreeningEvent,107


In [4]:
classes = np.unique(test_y).tolist()
test_y_numerical = [classes.index(x) for x in test_y]# tf.keras.utils.to_categorical([classes.index(x) for x in train_y])
test_y_categorical = tf.keras.utils.to_categorical(test_y_numerical)

Inference with keras model

In [5]:
model = tf.keras.models.load_model('bert_model/',custom_objects={'f1_score':f1_score,'recall':recall,'precision':precision})

2022-10-12 20:24:46.799301: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [6]:
tik = time() 
results = model.evaluate(test_x,test_y_categorical,batch_size=1)
tok = time() - tik

print("Average prediction time : {} secs".format(tok/len(test_x)))
print("Accuracy: ",results[1])
print("F-1 score: ",results[2])
print("Precision: ",results[3])
print("Recall: ",results[4])

700/700 [==============================] - 83s 118ms/step - loss: 0.2525 - categorical_accuracy: 0.9086 - precision: 0.8900 - recall: 0.8900 - f1_score: 0.8900
Average prediction time : 0.11911759751183645 secs
Accuracy:  0.9085714221000671
F-1 score:  0.8899999260902405
Precision:  0.8899999260902405
Recall:  0.8899999260902405
